In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score

from data_preprocessing import get_matrix

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def grid_search_cv(model, parameters, dataset, X):
    
    Y = dataset.label.values
    
    train_inds, test_inds = next(ShuffleSplit(test_size = 0.2, n_splits=2).split(X, Y))
    
    iterations = 5
    index = ShuffleSplit(n_splits = iterations, test_size = 0.2)
    
    clf = GridSearchCV(model, parameters, cv = index, scoring = 'accuracy', verbose=1)
    clf.fit(X[train_inds,:], Y[train_inds])
    
    best_model = clf.best_params_
    best_score_validation = clf.best_score_
    
    Y_pred = clf.predict(X[test_inds,:])
    accuracy = accuracy_score(Y[test_inds], Y_pred)
    
    return best_model, best_score_validation, accuracy

In [3]:
dataset = pd.read_csv('../data/Merged/spanish_dataset.csv')
dataset

,text,label
0,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",1
1,"La palabra ""haiga"", aceptada por la RAE La Rea...",1
2,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,1
3,UNAM capacitará a maestros para aprobar prueba...,0
4,Alerta: pretenden aprobar libros escolares con...,1
...,...,...
2566,"Recuperamos la historia de Aleixandra, la jove...",0
2567,"Reproches, tensión y sinceridad: la comida en ...",0
2568,"RT @ElMundoOpinion: ""PSOE, PP, Ciudadanos y Vo...",0
2569,Rusia cita al embajador español por unas decla...,0


In [4]:
matrix, df = get_matrix(data = dataset, representation = 'BoW', vocabulary_length = 5000, stemming = False, remove_stopwords = False, language = 'spanish')
print(matrix.shape)

(2571, 5000)


Random Forest Test

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
parameters = {'n_estimators': [100, 150],
             'max_features': [10]}

In [ ]:
grid_search_cv(model, parameters, df, matrix)